# Tarefa 03

- Leia os enunciados com atenção
- Saiba que pode haver mais de uma resposta correta
- Insira novas células de código sempre que achar necessário
- Em caso de dúvidas, procure os Tutores
- Divirta-se :)

In [3]:
import pandas as pd
import requests

####  1) Lendo de APIs
Vimos em aula como carregar dados públicos do governo através de um API (*Application Programming Interface*). No exemplo de aula, baixamos os dados de pedidos de verificação de limites (PVL) realizados por estados, e selecionamos apenas aqueles referentes ao estado de São Paulo.

1. Repita os mesmos passos feitos em aula, mas selecione os PVLs realizados por municípios no estado do Rio de Janeiro.
2. Quais são os três *status* das solicitações mais frequentes na base? Quais são suas frequências?
3. Construa uma nova variável que contenha o ano do **status**. Observe que ```data_status``` vem como tipo *object* no **DataFrame**. Dica: você pode usar o método ```.str``` para transformar o tipo da variável em string, em seguida um método como [**slice()**](https://pandas.pydata.org/docs/reference/api/pandas.Series.str.slice.html) ou [**split()**](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Series.str.split.html).
4. Indique a frequência de cada ano do campo construído no item (3).

In [20]:
# 1) Seu código aqui
url = 'https://apidatalake.tesouro.gov.br/ords/sadipem/tt/pvl'
params = {
    'uf':'RJ',
    'ente':'Município'
}

r = requests.get(url, params = params)
if r.status_code == 200:
    data = r.json()
    items = data.get('items', [])
    df = pd.DataFrame(items)
    print(df.head())
else:
    print(f"Erro na requisição: {response.status_code}")

   id_pleito tipo_interessado         interessado  cod_ibge  uf  \
0      27100        Município      Rio de Janeiro   3304557  RJ   
1      33609        Município           Vassouras   3306206  RJ   
2       5643        Município  São João de Meriti   3305109  RJ   
3       5914        Município           Nilópolis   3303203  RJ   
4       5267        Município      Paraíba do Sul   3303708  RJ   

                num_pvl                          status          num_processo  \
0  PVL02.002429/2017-78  Arquivado por decurso de prazo  17944.103668/2017-15   
1  PVL02.002481/2019-96                        Deferido  17944.102288/2019-17   
2                  None                       Arquivado  17944.000161/2010-26   
3                  None                    Regularizado  17944.000231/2012-16   
4                  None                    Regularizado  17944.000073/2014-58   

         data_protocolo                tipo_operacao  \
0  2018-07-31T18:50:09Z  Operação contratual interna  

In [26]:
# 2) Seu código aqui
top_status = df['status'].value_counts()
top_status.head()

status
Deferido                                                 120
Arquivado                                                 53
Encaminhado à PGFN com manifestação técnica favorável     50
Arquivado por decurso de prazo                            26
Regularizado                                              26
Name: count, dtype: int64

In [45]:
# 3) Seu código aqui
df["data_status"] = df["data_status"].astype(str)
df['ano_status'] = df['data_status'].str.slice(6)
print(df['ano_status'])

0      2019
1      2019
2      2010
3      2012
4      2023
       ... 
320    2023
321    2013
322    2008
323    2011
324    2013
Name: ano_status, Length: 325, dtype: object


In [46]:
# 4) Seu código aqui
frequencia = df['ano_status'].value_counts()
frequencia

ano_status
2008    39
2007    29
2023    28
2014    24
2012    24
2013    22
2010    22
2009    18
2019    16
2011    16
2020    15
2022    11
2015     8
2024     8
2002     8
2016     8
2025     7
2006     6
2017     5
2021     5
2018     4
2003     1
2004     1
Name: count, dtype: int64

####  2) Melhorando a interação com o API
Observe dois URLs de consultas diferentes, por exemplo o URL utilizado em aula, e o URL feito no exercício anterior. Compare-os e observe as diferenças.

1. Faça uma função em Python que recebe como argumento o UF da consulta e o tipo de interessado (```'Estado'```ou ```Município```), e que devolve os dados da consulta no formato *DataFrame*.
2. Quantas solicitações para o Estado podem ser consultadas para Minas Gerais com *status* em 'Arquivado por decurso de prazo' estão registradas?
3. Qual é o município da Bahia com mais solicitações deferidas?
4. Salve um arquivo .csv com os dados de solicitações da Bahia, com interessado = 'Estado'

In [48]:
#1) Seu código aqui
def consulta_pvl(uf, ente):
    params = {'uf': uf, 'ente': ente}
    
    response = requests.get(url, params=params)
    if response.status_code == 200:
        dados = response.json()['items']
        return pd.DataFrame(dados)
    else:
        raise Exception(f'Erro na requisição: {response.status_code}')

In [50]:
# 2) Seu código aqui
df_mg_estado = consulta_pvl('MG', 'estado')
filtro = df_mg_estado['status'] == 'Arquivado por decurso de prazo'
quantidade = df_mg_estado[filtro].shape[0]
print(quantidade)

306


In [72]:
# 3) Seu código aqui
df_ba_municipio = consulta_pvl('BA', 'Município')
df_deferidos = df_ba_municipio[df_ba_municipio['status'] == 'Deferido']
mais_deferidos = df_deferidos['interessado'].value_counts().head(2)
municipio_top = mais_deferidos.index[1]
quantidade = mais_deferidos.iloc[1]
print(f'O município com o maior número de solicitações deferidas é {municipio_top} com {quantidade}')

O município com o maior número de solicitações deferidas é Luís Eduardo Magalhães com 16


In [73]:
# 4) Seu código aqui
df_bahia_estado = consulta_pvl("BA", "Estado")
df_bahia_estado.to_csv("solicitacoes_bahia_estado.csv", index=False)